![](./img/chinahadoop.png)

# 歌曲序列建模
**[小象学院](http://www.chinahadoop.cn/course/landpage/15)《机器学习集训营》课程资料 by [@寒小阳](http://www.chinahadoop.cn/user/49339/about)**

我们来分析一下现在的场景，我们实际需要计算user或者item的相似度，协同过滤等neighborhood的方法是其中的一种。<br>
关于相近度这个问题，我们不是第一次遇见，我们在自然语言处理的机器学习问题中，需要把词映射成词向量，最简单的方式是one-hot，而为了达到更好的效果，我们通常需要捕捉一下近义词(比如“宾馆”和“酒店”)，我们会用到一个方法，叫做word2vec。

![](./img/word2vec.png)

```python
result = model.most_similar(u"习近平")
for e in result:
    print e[0], e[1]
```
```     
胡锦涛 0.809472680092
江泽民 0.754633367062
李克强 0.739740967751
贾庆林 0.737033963203
曾庆红 0.732847094536
吴邦国 0.726941585541
总书记 0.719057679176
李瑞环 0.716384887695
温家宝 0.711952567101
王岐山 0.703570842743
```

说起来word2vec，其实就是把词映射成一定维度的稠密向量，同时保持住词和词之间的关联性，主要体现在(欧式)距离的远近上。

那么问题来了，word2vec为什么能够学习到这样的结果？

因为我们相信“物以类聚，人以群分” “一个人的层次与他身边最近的一些人是差不多的”

同样的考虑，我们是不是可以认为，一个歌单里的歌曲，相互之间都有一定的关联性呢？就像句子中的词一样。答案是，是的！

咱们来写个程序跑一把。

## 从word2vec到song2vec

我们把歌曲的id序列取出来，类比于分完词后的句子，送到word2vec中去学习一下，看看会有什么效果。

In [1]:
#coding: utf-8
import multiprocessing
import gensim
import sys

def parse_playlist_get_sequence(in_line, playlist_sequence):
	song_sequence = []
	contents = in_line.strip().split("\t")
	# 解析歌单序列
	for song in contents[1:]:
		try:
			song_id, song_name, artist, popularity = song.split(":::")
			song_sequence.append(song_id)
		except:
			print "song format error"
			print song+"\n"
	playlist_sequence.append(song_sequence)


def train_song2vec(in_file, out_file):
	#所有歌单序列
	playlist_sequence = []
	#遍历所有歌单
	for line in open(in_file):
		parse_playlist_get_sequence(line, playlist_sequence)
	#使用word2vec训练
	cores = multiprocessing.cpu_count()
	print "using all "+str(cores)+" cores"
	print "Training word2vec model..."
	model = gensim.models.Word2Vec(sentences=playlist_sequence, size=150, min_count=3, window=7, workers=cores)
	print "Saving model..."
	model.save(out_file)

In [2]:
song_sequence_file = "./163_music_playlist.txt"
model_file = "./song2vec.model"
train_song2vec(song_sequence_file, model_file)

song format error
19169096:::

song format error
 Time to Say Goodbye (Con te partirò):::Sarah Brightman:::100.0

song format error
28935319:::

song format error
สั่น (Album Version):::Boy Sompob:::100.0

song format error
26902203:::What’s your name? (collaboration with 壇蜜)

song format error
:::SoulJa:::100.0

song format error
4967373:::テニスの王子様::ここからが・・・俺達II

song format error
:::V.A.:::5.0

song format error
28935319:::

song format error
สั่น (Album Version):::Boy Sompob:::100.0

song format error
376653:::野弧禅狂叱(宿香之战)

song format error
:::霹雳英雄:::5.0

song format error
374524:::赎?罪

song format error
赎罪岩:::霹雳英雄:::15.0

song format error
31563610:::

song format error
苍之礼赞:::花之祭P:::65.0

song format error
26902203:::What’s your name? (collaboration with 壇蜜)

song format error
:::SoulJa:::100.0

song format error
31838468:::Justiφ's(Tv Size Version)

song format error
:::佐藤和豊:::25.0

song format error
2924300:::

song format error
 Les Peulles Mortes (Autum Leaves):::Laura Fygi:::8

TypeError: 'NoneType' object is not iterable

模型已经训练完了，咱们来试一把预测，看看效果

这个预测的过程，实际上就是对某首歌曲，查找“最近”的歌曲（向量距离最近的歌曲）

In [5]:
import cPickle as pickle
song_dic = pickle.load(open("song.pkl","rb"))
model_str = "./song2vec.model"
model = gensim.models.Word2Vec.load(model_str)

In [6]:
song_id_list = song_dic.keys()[1000:1500:50]
for song_id in song_id_list:
    result_song_list = model.most_similar(song_id)

    print song_id, song_dic[song_id]
    print "\n相似歌曲 和 相似度 分别为:"
    for song in result_song_list:
        print "\t", song_dic[song[0]], song[1]
    print "\n"

29356670 にじいろのはる	ふわりP

相似歌曲 和 相似度 分别为:
	Aglamak Yok	Deniz Seki 0.379099398851
	知足（粤语版）（Cover 五月天）	阿细 0.356632173061
	Lifestyle	Joe Satriani 0.355613797903
	今昔幻想郷	まらしぃ 0.354855984449
	T・S・F in にっぽん!(Instrumental)	古川由利奈 0.353581249714
	Tokyo Operation	Dark Fount 0.35260361433
	爱到你发狂	谭咏麟 0.351889967918
	Pilgrimage	Terry Oldfield 0.351281344891
	Fortress (Radio Edit)	Ost & Meyer 0.349154919386
	Dreaming	Blue Lunar Monkey 0.346552073956


22677823 大阪ラプソディー	中澤裕子

相似歌曲 和 相似度 分别为:
	FRIENDSHIP	girl next door 0.396412461996
	아.미.고 (AMIGO: 아름다운 미녀를 좋아하면 고생한다)	SHINee 0.356553256512
	Red Omen	Two Steps From Hell 0.353555381298
	Consolação	Tenório Júnior E Seu Conjunto 0.347329318523
	First Meeting	Lightnin' Hopkins 0.345592081547
	めざまし 宁々	V.A. 0.344163119793
	Staendchen	Franz Schubert 0.343052297831
	Quiet Lake	Terry Oldfield 0.340119510889
	Love Is Bubble	BONNIE PINK 0.339400202036
	내가 사랑할 사람 (Piano ver.)	李仙姬 0.337993532419


28838650 想いのカナタ Game Size	Famishin

相似歌曲 和 相似度 分别为:
	Everyone Is Sleepin

## 进一步思考

所以我们用word2vec学会了哪些歌曲和哪些歌曲最接近。

我们来思考一些很现实同时又很难解决的问题。比如：
### 冷启动问题
我们经常会遇到冷启动的问题，比如没有任何信息的歌曲，我们如何对它做推荐呢？
* 如果是歌手发行的新歌曲，我们怎么进行推荐呢？
* 如果我听完(并收藏)了一首很冷门的歌，怎么进行推荐呢？

我们知道新歌(或者小众的歌)是非常难和其他的歌关联上的，我们有的信息太少了(很少有用户在它上面发生行为)。

1.1 一种解决办法当然是推荐热门的歌曲，但是其实没从个人兴趣出发，我们知道这并不是最好的办法，并没有太大的卵用。

1.2 我们把问题的粒度放粗一点，用同样的思路，比如一个可考虑的解决方案是，我们把**歌曲**的粒度上升到对应的**歌手**，把刚才的song_list替换成artist_list，重新用word2vec建模，这样我们可以得到和一个歌手最相关(接近)的歌手，再推荐这个歌手最热门的歌曲，相对1.1的方法针对性强一些。

### 用户兴趣预测问题
我们刚才完成的功能，类似酷狗音乐和网易音乐里针对一首歌的**“相似音乐”**，那么问题又来了，如果我们现在要对一个user用这套song2vec的方式推荐，我们怎么做呢？

* 每个人的兴趣都是有时效性的，这意味着说，3年前我喜欢王菲的歌，去年我喜欢五月天的歌，而今年我可能就改摇滚路线，喜欢汪峰的歌了。
* 每一首歌的热度也是不一样的，有一些热门的歌，如果用户能喜欢，当然是首选

那么，我们来做一个粗暴一点点的处理，把这2个维度拉进来，一起来针对一个用户做推荐。

**把每个用户喜欢(收藏)过的歌，沿着时间轴排好，同时由近到远给不同的衰减因子(比如最近一首歌是1，前一首是0.98，再前一首是0.98^2，以此类推...)，同时我们针对不同的歌曲热度，给定不同的推荐因子(比如热度100的是1，热度80的是0.9...)，每一首歌都可以拿回一个song2vec的推荐列表和对应的相似度，对相似度以时间衰减因子和热度权重进行加权，最后的结果排序后，展示给用户。**